# **Deep Research (DDG-Powered Agentic Pipeline)**

This project is a **small but complete agentic research pipeline** that plans web searches, gathers **high-signal snippets** via DuckDuckGo, and synthesizes a **long-form report** with inline citations.  

It also includes:
- A **Gradio UI** for interactive use.
- An optional **SendGrid email step** to deliver the final report as HTML.

---

## 🚀 **How It Works (Bird’s-Eye View)**

1. **`PlannerAgent`** — Analyzes your topic and proposes exactly **HOW_MANY_SEARCHES** focused queries, with clear reasoning for each.
2. **Search Agent (DuckDuckGo)** — Runs each query via a tool, producing compact summaries with a “Sources” list.
3. **`WriterAgent`** — Combines mini-summaries into a long report with inline citations `[1]`, `[2]`, etc., and a matching `final_citations` list.
4. **(Optional) Email Agent** — Sends the report in **nicely formatted HTML**.
5. **Gradio UI** — Streams progress and allows downloading the final Markdown.

---

## 🧠 **Agents at a Glance**

### **1. PlannerAgent — “What should I search?”**
- Generates **3 targeted search phrases** to cover:
  - Background
  - Trends
  - Data
  - Breadth
- Ensures maximum relevance and minimal overlap.

---

### **2. Search Agent (DuckDuckGo) — “Get me credible snippets quickly.”**
- Executes `ddg_search` **exactly once** per phrase.
- Summarizes in a few paragraphs.
- Appends a **small, credible Sources list**.

---

### **3. WriterAgent (Synthesizer) — “Turn everything into a coherent report.”**
- Builds a **multi-section Markdown report** (target length: **3–6k words**).
- Adds **inline citations** `[n]` that correspond to an **ordered final_citations** list.

---

### **4. Email Agent — “Send it.”**
- Uses **SendGrid** to deliver the final HTML-formatted report directly to recipients.

---

## 🎨 **User Experience**
- **Interactive Gradio UI** to visualize progress.
- Live streaming of agent outputs.
- **Downloadable Markdown** for offline use.
- Optional **email delivery** for easy sharing.


In [2]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool, ModelSettings
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import List, Dict, Any
from IPython.display import display, Markdown
import os, asyncio, textwrap
from typing import List, Dict, Optional, Any
from pydantic import BaseModel, Field
import gradio as gr
from ddgs import DDGS



c:\Users\saket\projects\deep_research_project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# # Fix TLS issues in some environments (VS Code / Windows / corp proxies)

import os, certifi
os.environ["SSL_CERT_FILE"] = certifi.where()  

import sendgrid

In [3]:
load_dotenv(override=True)

True

In [4]:
# ======================================================================================
# 1) DuckDuckGo tool
# ======================================================================================
ddg = DDGS()

# ---------- DDG tool ----------
@function_tool
def ddg_search(query: str,
               max_results: int = 10,
               region: str = "us-en",
               safesearch: str = "moderate") -> List[Dict[str, Any]]:
    """
    Run a DuckDuckGo search and return a list of results.
    Each result includes: title, url, snippet.
    """
    hits = ddg.text(
        query=query,
        max_results=max_results,
        region=region,
        safesearch=safesearch,
        verbose=True
    )
    results: List[Dict[str, Any]] = []
    for h in hits:
        url = h.get("href") or h.get("url") or ""
        if not url.startswith("http"):
            continue
        results.append({
            "title": h.get("title", "") or "",
            "url": url,
            "snippet": h.get("body") or h.get("snippet") or ""
        })
    return results

# ---------- Search agent (uses DDG tool) ----------
SEARCH_INSTRUCTIONS = (
    "You are a research assistant. Given a search term, call `ddg_search` exactly once to fetch results, "
    "then produce a elaborate summary in 4-5 paragraphs (>=1000 words). Capture only the key points—no fluff. "
    "Use the result titles and snippets only; do not browse further.\n\n"
    "After the summary, add a 'Sources' section listing 2–4 of the most relevant items as:\n"
    "[1] Title — URL\n[2] Title — URL\n"
)

search_agent = Agent(
    name="Search agent (DuckDuckGo)",
    instructions=SEARCH_INSTRUCTIONS,
    tools=[ddg_search],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)


In [5]:
# message = "Latest AI Agent frameworks in 2025"

# with trace("Search"):
#     result = await Runner.run(search_agent, message)

# display(Markdown(result.final_output))

The landscape of AI Agent frameworks in 2025 demonstrates a growing sophistication and diversity, reflecting the increasing demands of organizations for more advanced, autonomous systems. As AI technology matures, developers are compelled to choose from a variety of frameworks tailored to specific needs, particularly within the context of large language models (LLMs) and multi-agent systems. Understanding these frameworks is essential for staying competitive and driving innovation in the AI sector.

One of the most notable frameworks gaining traction is **LangGraph**, recognized for its ability to handle complex interactions in conversational AI. By integrating graph-based architecture, it allows for rich, contextual understanding, making it suitable for applications requiring intricate dialogue management. Alongside LangGraph, **DSPy** has emerged as a prominent choice among developers. This framework focuses on combining data programming with the LLM paradigm, allowing for flexible and efficient agent development that can adapt to a variety of input data configurations. As organizations seek to implement AI agents that can operate at scale, frameworks like DSPy are positioned as essential tools for integrating automation and machine learning seamlessly.

In the competitive landscape, **CrewAI** and **Microsoft's Semantic Kernel** cannot be overlooked. CrewAI offers a modular approach, allowing developers to tailor functionalities according to specific project requirements, thus accelerating development times while maintaining customizable features. The Semantic Kernel by Microsoft is particularly notable for its emphasis on leveraging existing natural language functionalities within Microsoft's ecosystem, which can simplify the integration of AI agents into enterprises already using Microsoft technologies. This cross-functional capability can reduce the barrier to entry for businesses looking to adopt AI solutions.

Furthermore, the **Microsoft AutoGen** framework stands out for its generative capabilities, specifically designed to create versatile agents that can handle generating multiple outputs from a single input query. This feature is particularly useful in applications ranging from content generation to customer support automation. **OpenAI Swarm** also represents a notable advancement, harnessing the power of multiple agents working collaboratively to tackle complex problems. Its architecture promotes efficient problem-solving by dividing tasks among agents, which improves processing time and enhances overall effectiveness.

A comprehensive understanding of these frameworks also requires a comparative analysis of their strengths and weaknesses. For instance, although LangGraph excels in dialogue complexity, it may lack the data handling capabilities of DSPy. Similarly, while Microsoft’s frameworks are well-suited for integration within existing infrastructures, developers may find certain functionalities more rigid compared to the flexibility offered by CrewAI. The decision around which framework to adopt hinges on specific project requirements, existing technology stacks, and the scalability needs of the organization.

As the AI agent landscape continues to diversify, developers are advised to consider not just the immediate capabilities of these frameworks, but also the broader ecosystem in which they will operate. The combination of features such as modularity, integration capability, and generative functions plays a crucial role in determining which framework will best serve an organization's long-term strategic goals. The emergence of collaboration-focused frameworks like OpenAI Swarm signals a shift toward cooperative AI solutions, which may become increasingly important as AI systems seek to work together in more complex and demanding environments.

In conclusion, the framework choices available for developing AI agents in 2025 are abundant, reflecting the rapid pace of technological advancement. The key players such as LangGraph, CrewAI, DSPy, and Microsoft’s various offerings underscore a shift towards more specialized and capable systems designed for diverse applications, from conversational interfaces to complex data handling. The evolution of these frameworks will continue to shape the future of AI, making it crucial for developers and organizations to remain informed and adaptable within this dynamic field.

### Sources
[1] Top 7 Frameworks for Building AI Agents in 2025 — https://www.analyticsvidhya.com/blog/2024/07/ai-agent-frameworks/  
[2] Best AI Agent Frameworks in 2025: Comparing LangGraph, DSPy ... — https://langwatch.ai/blog/best-ai-agent-frameworks-in-2025-comparing-langgraph-dspy-crewai-agno-and-more  
[3] A Detailed Comparison of Top 6 AI Agent Frameworks in 2025 — https://www.turing.com/resources/ai-agent-frameworks  
[4] Top AI Agent Frameworks 2025 — https://www.scalacode.com/blog/ai-agent-frameworks/  

In [6]:
# ======================================================================================
# 2) PlannerAgent — structured outputs (queries with reasons)
# ======================================================================================
HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."


class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [7]:

# message = "Latest AI Agent frameworks in 2025"

# with trace("Search"):
#     result = await Runner.run(planner_agent, message)
#     print(result.final_output)

searches=[WebSearchItem(reason='To find information on the newest AI agent frameworks released or popular in 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To gather insights on trends and advancements in AI agent technology for the year 2025.', query='trends in AI agents 2025'), WebSearchItem(reason='To look for specific frameworks that have gained traction or have been developed in 2025.', query='new AI frameworks 2025')]


In [9]:
# ======================================================================================
# 3) Email agent (optional)
# ======================================================================================


INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""


@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("garodisk@mail.uc.edu") # Change this to your verified email
    to_email = To("saketgarodia1@gmail.com") # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}



email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"), 
)


In [10]:
# ======================================================================================
# 4) WriterAgent (Synthesizer) — long-form report with citations
# ======================================================================================
# 
WRITER_INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive, deep report.\n"
    "You will be given the original query and a set of mini-summaries produced by assistants. "
    "Each mini-summary ends with a 'Sources' list. Use ONLY the information in those summaries and sources; "
    "do not invent facts.\n\n"
    "Tasks:\n"
    "1) Write an outline (headings only) for the report.\n"
    "2) Write a long-form Markdown report (aim 3,000–6,000 words) with clear headings/subheadings. "
    "Synthesize across the summaries, compare viewpoints, highlight agreements/disagreements, and avoid duplication. "
    "Where you reference specific facts, insert inline citations like [1], [2], etc. Map these to the ordered list "
    "of final citations you provide at the end.\n"
    "3) Provide a 3–5 sentence executive summary.\n"
    "4) Provide 3–6 follow-up research topics.\n\n"
    "Citation rules:\n"
    "- Only cite items that appear in the provided 'Sources' lists. Use the exact URLs.\n"
    "- Ensure the first time you cite a source in the body, it becomes [1], then [2], etc., in order of first mention. "
    "Your `final_citations` must match that order exactly. Please don't forget to cite sources in the end of the report\n"
)

class Citation(BaseModel):
    url: str = Field(description="Exact source URL used in the report")
    title: Optional[str] = Field(default=None, description="Page/article title, if known")

class ReportData(BaseModel):
    short_summary: str = Field(description="Executive summary (3–5 sentences).")
    outline_markdown: str = Field(description="Markdown outline (headings and bullets).")
    markdown_report: str = Field(description="Full long-form Markdown report with inline [n] citations.")
    follow_up_questions: List[str] = Field(description="3–6 suggested topics to research further.")
    final_citations: List[Citation] = Field(
        description="Ordered list of sources used (maps to [1..N])."
    )

writer_agent = Agent(
    name="WriterAgent (Synthesizer)",
    instructions=WRITER_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

In [11]:
# ======================================================================================
# 5) Small console helpers (pretty progress)
# ======================================================================================

RESET = "\x1b[0m"
BOLD  = "\x1b[1m"
DIM   = "\x1b[2m"
CYAN  = "\x1b[36m"
GREEN = "\x1b[32m"
YELL  = "\x1b[33m"
MAG   = "\x1b[35m"

def info(msg):    print(f"{CYAN}ℹ️  {msg}{RESET}")
def step(msg):    print(f"{GREEN}✅ {msg}{RESET}")
def warn(msg):    print(f"{YELL}⚠️  {msg}{RESET}")
def doing(msg):   print(f"{MAG}🔎 {msg}{RESET}")


In [12]:
# ======================================================================================
# 6) Orchestration helpers
# ======================================================================================

async def plan_searches(query: str) -> WebSearchPlan:
    result = await Runner.run(planner_agent, f"Query: {query}")
    plan = result.final_output
    # pretty print
    info("Planned search phrases:")
    for i, it in enumerate(plan.searches, 1):
        print(f"  {i}. {BOLD}{it.query}{RESET} {DIM}— {it.reason}{RESET}")
    return plan

async def search_one(item: WebSearchItem) -> str:
    # You can include the reason in the prompt for context, but the agent will call the tool.
    doing(f"Searching: {item.query}")
    input_msg = f"Search term: {item.query}\nReason: {item.reason}"
    result = await Runner.run(search_agent, input_msg, max_turns=4)
    step(f"Done: {item.query}")
    return result.final_output  # a Markdown 2–3 paragraph summary + Sources list

async def perform_searches(plan: WebSearchPlan) -> List[str]:
    # Run them concurrently
    tasks = [asyncio.create_task(search_one(it)) for it in plan.searches]
    return await asyncio.gather(*tasks)

def build_writer_input(original_query: str, mini_summaries: List[str]) -> str:
    # Plain text payload, no JSON. Clear delimiters.
    blocks = []
    blocks.append(f"Original query:\n{original_query}\n")
    blocks.append("=== MINI-SUMMARIES START ===")
    for i, s in enumerate(mini_summaries, 1):
        blocks.append(f"\n--- Summary {i} ---\n{s}\n")
    blocks.append("=== MINI-SUMMARIES END ===\n")
    return "\n".join(blocks)

async def write_report(original_query: str, mini_summaries: List[str]) -> ReportData:
    payload = build_writer_input(original_query, mini_summaries)
    result = await Runner.run(writer_agent, payload, max_turns=4)
    return result.final_output

In [13]:
# ======================================================================================
# 7) Actual Research
# ======================================================================================

async def run_research(query: str):
    with trace("Deep Research (DDG)"):
        print("Planning...")
        plan = await plan_searches(query)

        print("Searching...")
        mini_summaries = await perform_searches(plan)

        print("Synthesizing...")
        report = await write_report(query, mini_summaries)

        # Optional: await Runner.run(email_agent, report.markdown_report)
        return plan, mini_summaries, report

In [21]:
# ======================================================================================
# 8) Notebook helpers (pretty rendering of results)
# ======================================================================================


from IPython.display import Markdown, display

def show_report(report: ReportData, mini_summaries: List[str] | None = None):
    # Body
    display(Markdown(report.markdown_report))

    # References (if present)
    if getattr(report, "final_citations", None):
        lines = []
        for i, c in enumerate(report.final_citations, 1):
            title = (c.title or c.url).replace("[", r"\[").replace("]", r"\]")
            lines.append(f"{i}. [{title}]({c.url})")
        display(Markdown("### References\n\n" + "\n".join(lines)))
    else:
        display(Markdown("_No references returned by the writer._"))

def show_plan(plan: WebSearchPlan):
    rows = [f"{i}. **{it.query}** — {it.reason}" for i, it in enumerate(plan.searches, 1)]
    display(Markdown("### Planned search phrases\n\n" + "\n".join(rows)))

def show_mini_summaries(mini_summaries: List[str]):
    if not mini_summaries:
        display(Markdown("_No mini-summaries produced._"))
        return
    parts = ["### Mini-summaries"]
    for i, s in enumerate(mini_summaries, 1):
        parts.append(f"<details><summary><b>Summary {i}</b></summary>\n\n{s}\n\n</details>")
    display(Markdown("\n\n".join(parts)))


In [22]:
# ======================================================================================
# 9) Example usage (Jupyter: just `await main()`; script: `asyncio.run(main())`)
# ======================================================================================

plan, mini_summaries, report = await run_research("List of AI Agent frameworks  sorted by the top ones")
show_plan(plan)
show_mini_summaries(mini_summaries)
show_report(report, mini_summaries)


Planning...
ℹ️  Planned search phrases:
  1. top AI agent frameworks 2023 — To gather a comprehensive overview of popular AI agent frameworks currently available.
  2. best AI frameworks comparison reviews — To find comparisons, rankings, and user reviews of various AI frameworks to understand their strengths and weaknesses.
  3. emerging AI agent frameworks 2023 — To discover emerging or niche AI agent frameworks that might not be widely covered but are gaining traction.
Searching...
🔎 Searching: top AI agent frameworks 2023
🔎 Searching: best AI frameworks comparison reviews
🔎 Searching: emerging AI agent frameworks 2023
✅ Done: top AI agent frameworks 2023
✅ Done: best AI frameworks comparison reviews
✅ Done: emerging AI agent frameworks 2023
Synthesizing...


### Planned search phrases

1. **top AI agent frameworks 2023** — To gather a comprehensive overview of popular AI agent frameworks currently available.
2. **best AI frameworks comparison reviews** — To find comparisons, rankings, and user reviews of various AI frameworks to understand their strengths and weaknesses.
3. **emerging AI agent frameworks 2023** — To discover emerging or niche AI agent frameworks that might not be widely covered but are gaining traction.

### Mini-summaries

<details><summary><b>Summary 1</b></summary>

The landscape of AI agent frameworks in 2023 has evolved significantly, driven by the rapid advancements in artificial intelligence and the need for more capable, intuitive, and versatile systems. The exploration of these frameworks reveals a variety of options, each with unique features and applications suited to different industries and functions.

One of the standout frameworks is **LangChain**, noted as a modular orchestrator that has gained significant traction for its ability to chain tasks and facilitate interactions with large language models (LLMs) such as GPT-4. Its modular design allows developers to customize and extend functionality, making it an appealing choice for projects requiring tailored solutions. LangChain is particularly favored for its flexibility, allowing for integration with various APIs and data sources, thus enhancing its usability in diverse contexts ranging from simple chatbots to complex multi-agent systems.

Another rapidly emerging framework is **AutoGen**, developed by Microsoft. Launched in September 2023, AutoGen specializes in managing multi-agent conversations using an event-driven architecture. This framework has quickly amassed popularity, as demonstrated by its over 45,000 GitHub stars, suggesting that it resonates well with the developer community. AutoGen's capabilities are particularly highlighted in collaborative settings, where it outperforms traditional single-agent models on benchmarks like GAIA, making it a strong candidate for organizations seeking advanced AI solutions for interactive and collaborative tasks.

In addition to these, frameworks like **CrewAI** and **MetaGPT** are gaining attention for their innovative approaches to AI agent development. CrewAI focuses on enabling seamless collaboration among agents, facilitating workflows that require multiple AI agents to operate in harmony. Meanwhile, MetaGPT emphasizes adaptability and customizability, aiming to help developers create AI agents that can be tailored to specific business needs or application scenarios. These frameworks reflect the industry's shift towards more sophisticated AI systems that can learn and adapt over time, promoting a more autonomous operational paradigm.

Furthermore, the open-source community remains vital in this space, with frameworks such as **Auto-GPT** and **AgentGPT** demonstrating the benefits of collaborative development and shared innovation. These frameworks allow developers to tap into a wealth of community-driven improvements and insights, providing substantial resources for those looking to implement AI agents without the constraints of proprietary solutions. The active engagement of the open-source community ensures that these frameworks continue to evolve, keeping pace with emerging technologies and user requirements.

Overall, the choice of an AI agent framework in 2023 reflects broader trends in the industry towards increased connectivity, collaboration, and customization. The tools available are not merely functional; they enable organizations to rethink how they approach automation, decision-making, and user interaction through intelligent agents. As the field continues to grow, keeping abreast of developments in these frameworks will be essential for practitioners looking to leverage AI effectively in their respective domains.

### Sources
[1] The Best AI Agents in 2025: Tools, Frameworks, and Platforms Compared — https://www.datacamp.com/blog/best-ai-agents  
[2] 5 AI Agent Frameworks Compared — https://www.kdnuggets.com/5-ai-agent-frameworks-compared  
[3] Top AI Agent Frameworks in 2025: LangChain, AutoGen ... — https://medium.com/@iamanraghuvanshi/agentic-ai-3-top-ai-agent-frameworks-in-2025-langchain-autogen-crewai-beyond-2fc3388e7dec  
[4] Top AI Agent Frameworks in 2025 — https://www.curotec.com/insights/top-ai-agent-frameworks/

</details>

<details><summary><b>Summary 2</b></summary>

The search for "best AI frameworks comparison reviews" has yielded a variety of resources that delve deep into the characteristics and functionalities of leading AI frameworks as of 2025. A key takeaway is the ongoing expansion of AI frameworks, reflecting diverse needs spanning from simple machine learning to complex agent systems. These resources offer comprehensive reviews that encompass usability, features, benefits, and challenges associated with each framework, thereby assisting developers and businesses in making informed decisions about which frameworks to adopt.

**1. Overview of Leading AI Frameworks:**
The latest comparisons highlight several top AI agent frameworks such as LangGraph, LlamaIndex, CrewAI, Microsoft Semantic Kernel, and OpenAI Swarm. Each of these frameworks possesses distinct strengths tailored to specific requirements for AI solution development. For instance, LangGraph focuses on natural language processing while LlamaIndex emphasizes large-scale data processing capabilities. These frameworks have been evaluated based on several criteria, including scalability, adaptability, and community support. The workshop-style comparative guides provide not only rankings but also user testimonials reflecting real-world applications, which further bolster the decision-making process for prospective users.

**2. Open-Source vs. Commercial Options:**
The discussions surrounding AI frameworks also delve into the dichotomy between open-source and commercial platforms. Open-source frameworks are often highlighted for their flexibility and robust community support, allowing for extensive customization. On the other hand, commercial frameworks frequently tout enhanced features, support, and development resources. Articles such as "Top AI Frameworks & How To Choose The Right One" break this down extensively, guiding users through the pros and cons of each category. Additionally, frameworks such as TensorFlow and PyTorch are considered staples in the industry, attributed to their widespread adoption and comprehensive libraries. 

**3. Feature Highlights and Usabilities:**
The resources explore specific features that set frameworks apart. For instance, Microsoft's Semantic Kernel integrates natural language understanding with other AI capabilities, making it suitable for complex applications that require semantic comprehension. In contrast, frameworks like AutoGen are tailored for rapid deployment and user interaction scenarios. Articles from sources like Turing and BairesDev detail these capabilities, allowing readers to assess which aspects are crucial for their specific AI implementations. Real-world case studies demonstrate how enterprises leverage these frameworks for tasks ranging from predictive analytics to automation in operations, showcasing practical applications that inform potential users’ choices.

**4. Recommendation Breakdown:**
The various articles recommend frameworks based on the use case, expertise level, and organizational needs. For example, a company looking for a versatile framework for deep learning projects may lean towards TensorFlow, while another that requires quick deployment and ease of use might prefer an agent framework like CrewAI. Webinars, tech talks, and detailed blog posts are often referenced as supportive materials for further research, emphasizing the influence of active community engagement and ongoing development in framework selection. This engagement often leads to prompt updates and innovations, keeping these tools at the cutting edge of AI technology.

In summary, veterans and newcomers alike to AI development can find valuable insights in these reviews on the best AI frameworks available. They present a detailed comparison that not only ranks these frameworks but also contextualizes their capabilities within current technological trends. For businesses and developers looking to implement AI solutions, understanding these dynamics can significantly enhance both the effectiveness of their applications and their strategic planning.

**Sources:**
[1] Top 9 AI Agent Frameworks as of July 2025 — https://www.shakudo.io/blog/top-9-ai-agent-frameworks  
[2] Top AI Frameworks & How To Choose The Right One — https://lakefs.io/blog/ai-frameworks/  
[3] AI Agent Frameworks: A Detailed Comparison — https://www.turing.com/resources/ai-agent-frameworks  
[4] Top AI Frameworks in 2025: A Review — https://www.bairesdev.com/blog/ai-frameworks/  

</details>

<details><summary><b>Summary 3</b></summary>

The landscape of AI agent frameworks is currently experiencing significant evolution, driven by advancements in artificial intelligence and the demand for more sophisticated automation tools. The proliferation of various AI agent frameworks is enabling developers to build solutions tailored for an array of applications, from simple task automation to complex decision-making systems. As described in the article "Exploring Emerging AI Agent Frameworks," the ecosystem features tools accommodating different levels of sophistication, offering everything from lightweight libraries suitable for beginners to comprehensive multi-agent systems that allow for intricate interactions among several AI components. This diversity indicates not just rapid technological advancements but also a shift toward creating scalable and collaborative AI systems.

Among the notable developments in AI agent frameworks is the rise of specialized agents in the decentralized finance (DeFi) sector, as highlighted by the article "AI Agents: Emergence of Lisan al Gaib." These frameworks aim to simplify and automate the complex financial tasks associated with DeFi operations, a field that has traditionally been daunting for average users due to its complexity. This new category of AI agents integrates seamlessly into financial ecosystems, potentially minimizing barriers to entry and enhancing user experience by providing intelligent solutions tailored to the specific needs of users engaged in decentralized applications.

In addition to DeFi applications, some emerging frameworks focus on orchestrating agentic networks. According to "Beyond the AI Hype: The Reality of Agentic Networks," these agentic frameworks are designed to optimize resource use, making heavy-weight open-source applications much more efficient. Such architectures offer distinct advantages by managing various agents collaboratively, enabling a more efficient allocation of tasks and resources. This leads to a situation in which multiple AI agents can work concurrently, significantly speeding up processes that would traditionally be handled sequentially by single agents.

Furthermore, advancements in large language models (LLMs) are bolstering the emergence of more capable AI agents, leading to what's termed the "Agentic Web." This paradigm shift, detailed in the article "Agentic Web: Weaving the Next Web with AI Agents," points toward a future where AI agents engage in goal-driven interactions directly with each other. In this model, agents are not just passive tools controlled by humans, but active participants capable of initiating actions based on pre-set objectives, collaborating on tasks autonomously, and making decisions in real-time. This transformation marks a fundamental change in how agents interact with the digital landscape, moving from mere assistance tools to more autonomous entities that can learn and adapt independently.

Notably, frameworks such as CrewAI are coming to the forefront, as explored in "AI Agent Frameworks Explained: Key Features and Top Choices." These frameworks focus on multi-agent collaboration, which is key to achieving complex tasks efficiently. CrewAI, in particular, facilitates cooperation among AI agents operating on different aspects of a task, thereby advancing collective problem-solving capabilities in various industries, from tech to healthcare. By fostering collaborative efficiencies, these frameworks hold the promise of significantly enhancing productivity and innovation through AI-driven initiatives.

Additionally, the broader scope of AI agents encompasses not only the needs of financial technologies but also sectors like healthcare, automation, and even general business applications. The article "From Tools to Teammates: The Rise of Autonomous AI Agents" illustrates how autonomous AI agents are being designed to achieve specific goals through perception, decision-making, and execution. These components, including machine learning models and logic frameworks, allow agents to gather information, process it, and make informed decisions, effectively functioning as capable teammates rather than mere tools.

As developers continue to explore the breadth of AI agent frameworks, the focus is on creating solutions that are not only efficient and powerful but also user-friendly and intuitive. The diversity of frameworks now available offers developers a plethora of choices tailored to their specific needs, enhancing creativity and fostering innovation. With ongoing advancements and increased adoption across multiple industries, it becomes clear that AI agent frameworks are set to redefine how intelligent systems are built and interacted with.

In conclusion, the evolution of AI agent frameworks in 2023 showcases a significant shift towards more intelligent, autonomous, and collaborative systems. From their application in financial technologies to healthcare and beyond, emerging frameworks signal a new phase in how AI is integrated into daily operations, shaping the future of technology and its impact on multiple sectors.

### Sources
[1] Exploring Emerging AI Agent Frameworks — [Link](https://www.linkedin.com/pulse/exploring-emerging-ai-agent-frameworks-hitesh-sarda-k0gtc)  
[2] AI Agents: Emergence of Lisan al Gaib — [Link](https://www.gatenode.org/learn/articles/ai-agents-emergence-of-lisan-al-gaib/6568)  
[3] Agentic Web: Weaving the Next Web with AI Agents — [Link](https://arxiv.org/html/2507.21206v1)  
[4] CrewAI: AI Agent Frameworks Explained — [Link](https://www.webmobinfo.ch/blog/ai-agent-frameworks-for-businesses)

</details>

# Report on AI Agent Frameworks in 2023

## 1. Introduction
The landscape of AI agent frameworks is rapidly evolving, necessitated by advancements in artificial intelligence and increasing demands for sophisticated automation tools. This report aims to explore the most prominent AI agent frameworks in 2023, evaluating their unique features, applications, and the contexts in which they excel. Furthermore, it will discuss the dichotomy between open-source and commercial frameworks while providing insights into future trends within the industry.

## 2. Overview of AI Agent Frameworks
AI agent frameworks serve as essential tools that enable developers to build intelligent systems suitable for varying tasks—from simple automation to complex decision-making processes. The rapid expansion of these frameworks reflects a diverse range of needs within the AI ecosystem, emphasizing usability, adaptability, and community engagement as critical factors influencing their adoption. The necessity for tailored solutions has positioned frameworks like LangChain, AutoGen, and CrewAI at the forefront of this evolution, addressing the varying complexities of user requirements.

## 3. Leading AI Agent Frameworks
### LangChain  
LangChain stands out as a modular orchestrator that excels in chaining tasks and facilitating interactions with large language models (LLMs) such as GPT-4. Its flexible architecture allows developers to integrate diverse APIs and data sources, making it a popular choice for applications ranging from chatbots to multi-agent systems. LangChain’s adaptability helps organizations rethink automation and user interactions through intelligent agents, thereby enhancing overall productivity and engagement [3].  

### AutoGen  
Developed by Microsoft, AutoGen has gained significant traction since its launch in September 2023 due to its event-driven architecture for managing multi-agent conversations. With over 45,000 GitHub stars, it has emerged as a reliable choice for organizations looking for advanced collaborative solutions. Notably, AutoGen has demonstrated superior performance in collaborative benchmarks like GAIA, providing substantial advantages over traditional single-agent models [2][3].  

### CrewAI  
CrewAI emphasizes seamless collaboration amongst agents, designed to enable workflows where multiple AI agents operate in harmony. This collaborative feature is particularly valuable for enterprises looking to leverage collective problem-solving capabilities across diverse sectors, showcasing its effectiveness in enhancing productivity and fostering innovation [3][4].  

### MetaGPT  
MetaGPT prioritizes adaptability and customizability, making it a compelling option for developers needing to tailor AI agents to specific business applications or scenarios. Its focus on user-friendly design and flexibility has allowed it to remain relevant in the fast-paced AI environment, attracting businesses that seek specialized solutions [1].  

### Other Notable Frameworks  
In addition to the frameworks mentioned, others like LangGraph, LlamaIndex, Microsoft Semantic Kernel, and OpenAI Swarm are gaining recognition. These frameworks offer various capabilities from natural language processing to large-scale data manipulation, allowing developers to select the most suitable tools based on their specific needs and applications [2].  

## 4. Open-Source vs. Commercial Frameworks
The discussion surrounding AI frameworks often addresses the benefits and challenges associated with open-source versus commercial options. Generally, open-source frameworks provide extensive customization and lower entry costs, being driven by community support and rapid innovation. On the other hand, commercial frameworks typically offer enhanced features, resources, and dedicated support, making them appealing for organizations with greater investment capabilities.

Frameworks like TensorFlow and PyTorch remain staples in the industry, widely acknowledged for their comprehensive libraries and ease of use, catering to a broad audience from beginners to AI specialists. Furthermore, frameworks such as Auto-GPT and AgentGPT illustrate the advantages of collaborative development, as they continually evolve through community contributions [1][2].  

## 5. Specific Features and Use Cases
AI agent frameworks are being crafted to meet various needs across domains. For instance, the Microsoft Semantic Kernel integrates natural language understanding with multi-faceted AI capabilities, allowing it to cater to complex applications requiring deep semantic comprehension [2]. In comparison, AutoGen’s design targets rapid deployment for user interaction scenarios [2].  

Additionally, specific frameworks are emerging as leaders in niche domains such as decentralized finance (DeFi), where specialized agents simplify complex financial tasks, enabling users to engage more readily with this sector. Such frameworks hold potential for significant market disruption by automating intricate operations that were previously cumbersome [3].  

## 6. Future Trends in AI Agent Frameworks
The evolution of AI agent frameworks indicates a shift toward autonomous and collaborative systems—characteristics of the emerging 'Agentic Web.' As frameworks increasingly allow agents to interact independently and pursue set objectives, operations across various applications are becoming more intricate and efficient. This paradigm is indicative of a future where AI agents may collaborate not only with humans but also with each other to drive productivity [3].  

Emerging frameworks, such as CrewAI and AutoGen, aim to facilitate these advancements by enabling cooperative behaviors among agents across multiple applications and industries, thereby fostering innovation and enhancing outcomes [1][4].  

## 7. Recommendations for Choosing AI Frameworks
Choosing the right AI agent framework hinges on several factors, including industry requirements, scalability, adaptability, and ease of use. For organizations seeking robust solutions for deep learning projects, frameworks like TensorFlow are recommended, while those looking for rapid, user-friendly deployment may prefer CrewAI or AutoGen. Engaging with community insights, testimonials, and expert opinions can significantly inform the decision-making process for prospective users [4][2].  

## 8. Conclusion
In conclusion, the landscape of AI agent frameworks in 2023 is marked by a significant push towards creating intelligent, autonomous, and collaborative systems. With rapid technological advancements enabling diverse applications across multiple industries, practitioners must stay abreast of evolving frameworks to leverage AI effectively within their domains. As organizations increasingly embrace these intelligent technologies, the role of AI agent frameworks in shaping future innovations cannot be overstated [1][3][4].

## 9. Follow-Up Research Topics
1. Exploring the impact of AI agent frameworks on industry-specific applications.
2. Comparative studies between large language models and different AI agent frameworks.
3. Investigating the role of community engagement in the evolution of open-source AI frameworks.
4. Analyses of use cases for advanced collaborative frameworks in business settings.
5. Researching future trends in AI capabilities and their implications for the workforce.

### References

1. [Top AI Agent Frameworks in 2025: LangChain, AutoGen ...](https://medium.com/@iamanraghuvanshi/agentic-ai-3-top-ai-agent-frameworks-in-2025-langchain-autogen-crewai-beyond-2fc3388e7dec)
2. [5 AI Agent Frameworks Compared](https://www.kdnuggets.com/5-ai-agent-frameworks-compared)
3. [Top 9 AI Agent Frameworks as of July 2025](https://www.shakudo.io/blog/top-9-ai-agent-frameworks)
4. [Top AI Frameworks in 2025: A Review](https://www.bairesdev.com/blog/ai-frameworks/)

In [ ]:
# ======================================================================================
# 10) Email test
# ======================================================================================


from agents import Runner
import html

query = "List of AI Agent frameworks in 2025"
subject = f"Deep Research – {query}"

html_body = f"""
<h1>{html.escape(subject)}</h1>
<hr/>
<pre style="white-space:pre-wrap;font-family:ui-monospace, SFMono-Regular, Menlo, Consolas;">
{html.escape(report.markdown_report)}
</pre>
"""

await Runner.run(
    email_agent,
    f"""Send this as an HTML email using your tool.
subject: {subject}
html_body:
<<<HTML
{html_body}
HTML"""
)


RunResult(input='Send this as an HTML email using your tool.\nsubject: Deep Research – List of AI Agent frameworks in 2025\nhtml_body:\n<<<HTML\n\n<h1>Deep Research – List of AI Agent frameworks in 2025</h1>\n<hr/>\n<pre style="white-space:pre-wrap;font-family:ui-monospace, SFMono-Regular, Menlo, Consolas;">\n# The Evolving Landscape of AI Agent Frameworks in 2025\n\n## Introduction\nAs the world moves into 2025, the landscape of AI agent frameworks has experienced a substantial evolution. Organizations are increasingly investing in intelligent automation solutions to enhance productivity and improve decision-making processes. This report delves into the key AI agent frameworks expected to dominate in 2025, examines technological trends influencing this evolution, and assesses the implications for businesses and various industries.\n\n## Evolution of AI Agent Frameworks\nThe advancement of AI frameworks has been transformative, with several frameworks emerging that cater to specific or

In [57]:

# ======================================================================================
# 11) Gradio demo (optional)
# ======================================================================================


"""
Gradio UI for your Deep Research pipeline.

This app assumes you already have the orchestration helpers defined in the same Python
process (plan_searches, perform_searches, write_report, run_research) or imported from
your module. If you ran your earlier code in the same session (e.g., Jupyter), this
file can import them directly with a `from your_module import run_research`.

If you prefer a fully-standalone app, see the STANDALONE NOTE near the bottom for
how to swap in an import.
"""

import asyncio
import os
from datetime import datetime
from typing import List, Tuple

import gradio as gr

try:
    run_research  # type: ignore[name-defined]
except NameError:
    raise RuntimeError(
        "run_research(query) is not defined in this process.\n"
        "Import it here, e.g.: from deep_research import run_research\n"
        "Or run this app in the same kernel where you defined run_research."
    )


def _plan_to_rows(plan) -> List[list]:
    """Turn WebSearchPlan into rows for a DataFrame-friendly structure (list of lists)."""
    rows: List[list] = []
    try:
        for i, it in enumerate(plan.searches, 1):
            rows.append([i, getattr(it, "query", ""), getattr(it, "reason", "")])
    except Exception:
        pass
    return rows


def _citations_to_md(report) -> str:
    """Render final citations as Markdown numbered list."""
    if not getattr(report, "final_citations", None):
        return ""
    lines = ["\n### Sources\n"]
    for idx, c in enumerate(report.final_citations, 1):
        title = getattr(c, "title", None) or "Source"
        url = getattr(c, "url", "")
        if url:
            lines.append(f"{idx}. [{title}]({url})")
        else:
            lines.append(f"{idx}. {title}")
    return "\n".join(lines)


def _write_markdown_to_file(md: str, folder: str = ".") -> str:
    ts = datetime.now().strftime("%Y%m%d-%H%M%S")
    path = os.path.join(folder, f"deep_research_{ts}.md")
    with open(path, "w", encoding="utf-8") as f:
        f.write(md)
    return path


async def stream_research(chat_history: List[Tuple[str, str]], topic: str, progress=gr.Progress(track_tqdm=False)):
    """
    Chat-like streaming handler. Yields intermediate updates as tuples compatible with Chatbot(type="tuples").
    """
    history = list(chat_history or [])
    history.append((topic, "Starting deep research..."))
    yield history, [], "", "", "", None

    progress(0.05, desc="Planning searches…")
    history[-1] = (topic, "Planning searches…")
    yield history, [], "", "", "", None

    progress(0.12, desc="Running pipeline… (this may take a bit)")
    plan, mini_summaries, report = await run_research(topic)  # type: ignore[name-defined]

    progress(0.55, desc="Formatting results…")
    plan_rows = _plan_to_rows(plan)
    history[-1] = (topic, "Searches planned. Gathering and synthesizing…")
    yield history, plan_rows, "", "", "", None

    if mini_summaries:
        parts = ["## Mini-summaries"]
        for i, s in enumerate(mini_summaries, 1):
            parts.append(f"<details><summary><b>Summary {i}</b></summary>\n\n{s}\n\n</details>")
        mini_md = "\n\n".join(parts)
    else:
        mini_md = "(No mini-summaries produced)"

    yield history, plan_rows, mini_md, "", "", None

    report_md = getattr(report, "markdown_report", "")
    sources_md = _citations_to_md(report)

    full_md = report_md + ("\n\n" + sources_md if sources_md else "")
    file_path = _write_markdown_to_file(full_md)

    history[-1] = (topic, "Done ✅")
    progress(1.0, desc="Complete")
    yield history, plan_rows, mini_md, report_md, sources_md, file_path


with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 🔎 Deep Research (Gradio)
        Enter a topic and I'll plan searches, gather snippets, and synthesize a long-form report.
        """
    )

    with gr.Row():
        chatbot = gr.Chatbot(label="Research Chat", height=350, type="tuples")
    
    with gr.Row():
        topic = gr.Textbox(placeholder="e.g., Latest AI Agent frameworks in 2025", label="Topic", lines=1)
        go = gr.Button("Run Research", variant="primary")

    with gr.Row():
        plan_table = gr.Dataframe(headers=["#", "query", "reason"], label="Planned web searches", interactive=False)

    mini_md = gr.Markdown(label="Mini-summaries")
    report_md = gr.Markdown(label="Synthesis report (Markdown)")
    sources_md = gr.Markdown(label="Sources")

    with gr.Row():
        dl = gr.DownloadButton(label="Download Markdown", value=None)

    state = gr.State([])

    go.click(
        fn=stream_research,
        inputs=[state, topic],
        outputs=[chatbot, plan_table, mini_md, report_md, sources_md, dl],
        show_progress=True,
    ).then(lambda h: h, inputs=chatbot, outputs=state)

    gr.Examples([
        ["Latest AI Agent frameworks in 2025"],
        ["State of multimodal RAG for e-commerce"],
        ["Carbon accounting tools for utilities in 2025"]
    ], inputs=[topic])

if __name__ == "__main__":
    demo.launch(share = True)

"""
STANDALONE NOTE:
Replace the `try/except` block with:

    from your_module import run_research

Ensure that `your_module.run_research` matches the signature used above.
"""


C:\Users\saket\AppData\Local\Temp\ipykernel_16316\1750919685.py:114: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Research Chat", height=350, type="tuples")


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://254527a4e3d76e7424.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


'\nSTANDALONE NOTE:\nReplace the `try/except` block with:\n\n    from your_module import run_research\n\nEnsure that `your_module.run_research` matches the signature used above.\n'

Planning...
ℹ️  Planned search phrases:
  1. AI alignment research India — To find the current status and research related to AI alignment in India.
  2. AI ethics initiatives India — To identify organizations and initiatives focused on AI ethics and alignment in India.
  3. AI alignment challenges India — To explore discussions, forums, and papers specifically about AI alignment challenges faced in the Indian context.
Searching...
🔎 Searching: AI alignment research India
🔎 Searching: AI ethics initiatives India
🔎 Searching: AI alignment challenges India
✅ Done: AI alignment research India
✅ Done: AI ethics initiatives India


Trace already exists. Creating a new trace, but this is probably a mistake.


✅ Done: AI alignment challenges India
Synthesizing...
Planning...
ℹ️  Planned search phrases:
  1. mechanical engineering jobs in Chicago — To find specific job listings and opportunities in the field of mechanical and energy engineering in Chicago.
  2. energy engineering jobs in Chicago — To gather insights on the energy engineering job market and demand in Chicago.
  3. companies hiring mechanical energy engineers in Chicago — To identify companies and organizations in Chicago that hire mechanical and energy engineers.
Searching...
🔎 Searching: mechanical engineering jobs in Chicago
🔎 Searching: energy engineering jobs in Chicago
🔎 Searching: companies hiring mechanical energy engineers in Chicago
✅ Done: mechanical engineering jobs in Chicago
✅ Done: companies hiring mechanical energy engineers in Chicago


Trace already exists. Creating a new trace, but this is probably a mistake.


✅ Done: energy engineering jobs in Chicago
Synthesizing...
